In [1]:
#Names of Group Members

In [23]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup

# Q1.) Web Crawling Tables

### Q1.A.) Create a list of links for all the wikipedia pages for NYSE traded companies A-Z and 0-9

In [3]:
# Set up user agent
user_agent_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246" 
,"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36 "
,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9 "
,"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1" 
,"Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"] 

headers = {
    'User-Agent': user_agent_list[4]}

In [4]:
URL = "https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange"
html = requests.get(URL, headers = headers).content
soup = BeautifulSoup(html,"html.parser")

In [5]:
# Calculate Lenth of String
x = "Companies listed on the New York Stock Exchange"
len(x)

47

In [6]:
# First, get all the website that contains stock info
base_url = 'https://en.wikipedia.org'
links_all = []
all_a = soup.find_all('a',href = True)
for a in all_a:
    if a.text[:48] == "Companies listed on the New York Stock Exchange ":
        links_all.append(base_url + a['href'])

links_all

['https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(0%E2%80%939)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(A)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(B)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(C)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(D)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(E)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(F)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(G)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(H)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(I)',
 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_(J)',
 'https://en.wikipedia.org/wiki/Companies_li

In [7]:
# Second, get through each url to extract the company wikipedia
company_url = []
for url in links_all:
    page = requests.get(url,headers = headers)
    soup = BeautifulSoup(page.content,'html.parser')
    links = soup.findAll('a',href = True, title = True)
    i = url[-2]
    for a in links:
        title = a.get('title')
        if (title == a.text[0:len(title)] and title[:1] == i) or title[0].isdigit():
            company_url.append(base_url + a["href"])
            
company_url

['https://en.wikipedia.org/wiki/3D_Systems',
 'https://en.wikipedia.org/wiki/3M',
 'https://en.wikipedia.org/wiki/8x8',
 'https://en.wikipedia.org/wiki/A._O._Smith',
 'https://en.wikipedia.org/wiki/A10_Networks',
 'https://en.wikipedia.org/wiki/AAR_Corp',
 'https://en.wikipedia.org/wiki/Abbott_Laboratories',
 'https://en.wikipedia.org/wiki/AbbVie',
 'https://en.wikipedia.org/wiki/Abercrombie_%26_Fitch',
 'https://en.wikipedia.org/wiki/ABM_Industries',
 'https://en.wikipedia.org/wiki/Accenture',
 'https://en.wikipedia.org/wiki/ACCO_Brands',
 'https://en.wikipedia.org/wiki/Actuant',
 'https://en.wikipedia.org/wiki/Acuity_Brands',
 'https://en.wikipedia.org/wiki/Adams_Express_Company',
 'https://en.wikipedia.org/wiki/Argentina',
 'https://en.wikipedia.org/wiki/Adeptus_Health',
 'https://en.wikipedia.org/wiki/Adient',
 'https://en.wikipedia.org/wiki/Advance_Auto_Parts',
 'https://en.wikipedia.org/wiki/AdvanSix',
 'https://en.wikipedia.org/wiki/Advantage_Oil_%26_Gas',
 'https://en.wikipedia

### Q1.B.) Crawl through all the URLs and make 1 DF with all the NYSE publically traded companies

In [8]:
df_list = []
for url in links_all:
    df = pd.read_html(url)
    df_list.append(df[1])

# Concatenate the list of DataFrames into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)
final_df

,Stock name,Symbol,Country of origin
0,3D Systems Corporation,DDD,US
1,3M Company,MMM,US
2,500.com,WBAI,China
3,58.com Inc.,WUBA,China
4,8x8 Inc.,EGHT,US
...,...,...,...
2718,Zimmer Biomet,ZBH,United States
2719,Zions Bancorporation,ZION,United States
2720,Zuora,ZUO,United States
2721,Zoetis,ZTS,United States


### Q1.C.) What is the percetages of companies that contain 1 letter, 2 letters, 3 letters, 4 letters and 5 letters in the ticker?

In [9]:
# Total companies count
total = len(final_df['Symbol'])

In [10]:
final_df['ticker'] = final_df['Symbol'].apply(len)
final_df.head()

,Stock name,Symbol,Country of origin,ticker
0,3D Systems Corporation,DDD,US,3
1,3M Company,MMM,US,3
2,500.com,WBAI,China,4
3,58.com Inc.,WUBA,China,4
4,8x8 Inc.,EGHT,US,4


In [11]:
# 1 Letter Percentage
one_per = round((final_df['Symbol'].apply(len)==1).sum()/total*100,2)
print(f"{one_per:.2f}% of companies contain 1 letter in the ticker.")

0.88% of companies contain 1 letter in the ticker.


In [12]:
# 2 Letter Percentage
two_per = round((final_df['Symbol'].apply(len)==2).sum()/total*100,2)
print(f"{two_per:.2f}% of companies contain 1 letter in the ticker.")

6.98% of companies contain 1 letter in the ticker.


In [13]:
# 3 Letter Percentage
three_per = round((final_df['Symbol'].apply(len)==3).sum()/total*100,2)
print(f"{three_per:.2f}% of companies contain 1 letter in the ticker.")

65.19% of companies contain 1 letter in the ticker.


In [14]:
# 4 Letter Percentage
four_per = round((final_df['Symbol'].apply(len)==4).sum()/total*100,2)
print(f"{four_per:.2f}% of companies contain 1 letter in the ticker.")

14.84% of companies contain 1 letter in the ticker.


In [15]:
# 5 Letter Percentage
five_per = round((final_df['Symbol'].apply(len)==5).sum()/total*100,2)
print(f"{five_per:.2f}% of companies contain 1 letter in the ticker.")

0.81% of companies contain 1 letter in the ticker.


# Q2.) Web Scraping Using Beautiful Soup

### Q2.A.) Using Beautiful soup .findAll method you will webscrape the front page of Reddit. Get a list of all of the "timestamps"

In [16]:
URL = "https://www.reddit.com"
html = requests.get(URL, headers = headers).content
soup = BeautifulSoup(html,"html.parser")

In [17]:
time_list = []
for a in soup.find_all('span',{'data-click-id' : "timestamp"}):
    time_list.append(a.text)
    
time_list

['15 hours ago',
 '17 hours ago',
 '3 hours ago',
 '6 hours ago',
 '16 hours ago',
 '12 hours ago',
 '6 hours ago']

### Q2.B.) Using the functions findChild, descendents, etc. locate the post title and store in a list.

In [18]:
title = []
divs = soup.find_all('div', {'class':"_2SdHzo12ISmrC8H86TgSCp _3wqmjmv3tb_k-PROt7qFZe",'style':"--posttitletextcolor:#222222"})
for div in divs:
    title.append(div.findChild().text)

title

["Bernie Sanders Says US Should Confiscate 100% Of Any Money Americans Make Above $999M: 'They Can Survive Just Fine' - what do you think ?",
 'Skip the presets. Make it your own with Blend modes. With Photoshop, everyone can.',
 'This woman thought she could steal over $2,000 in meat in broad daylight',
 'My dad does this to avoid cutting pepperoni',
 'What other games do this?',
 'My husband says I’m a psychopath',
 'Books that irritated you',
 'I have a meeting scheduled with my boss because I napped on my 30 min break']

### Q2.C.) Create a dataframe that has the associated title and post time for each post.

In [19]:
title_df = pd.DataFrame({'Title':title[:-1],'TimeStemp':time_list})
title_df

,Title,TimeStemp
0,Bernie Sanders Says US Should Confiscate 100% ...,15 hours ago
1,Skip the presets. Make it your own with Blend ...,17 hours ago
2,"This woman thought she could steal over $2,000...",3 hours ago
3,My dad does this to avoid cutting pepperoni,6 hours ago
4,What other games do this?,16 hours ago
5,My husband says I’m a psychopath,12 hours ago
6,Books that irritated you,6 hours ago


# Q3.) RegEx

### Q3.A.) Using RegEx, get all the urls of ladder faculty profiles for UCLA Economics

In [20]:
URL = "https://economics.ucla.edu/faculty/ladder"
html = requests.get(URL, headers = headers).content
soup = BeautifulSoup(html,"html.parser")

In [21]:
# Using re compile
links = soup.find_all('a', href=re.compile(r'https://economics.ucla.edu/person/'))
prof_urls = [link['href'] for link in links]

# Get Unique Value
prof_urls = list(set(prof_urls))

prof_urls

['https://economics.ucla.edu/person/adriana-lleras-muney/',
 'https://economics.ucla.edu/person/oleg-itskhoki/',
 'https://economics.ucla.edu/person/rosa-liliana-matzkin/',
 'https://economics.ucla.edu/person/alexander-bloedel/',
 'https://economics.ucla.edu/person/denis-chetverikov/',
 'https://economics.ucla.edu/person/lee-e-ohanian/',
 'https://economics.ucla.edu/person/bernardo-s-silveira/',
 'https://economics.ucla.edu/person/michela-giorcelli/',
 'https://economics.ucla.edu/person/simon-board/',
 'https://economics.ucla.edu/person/hugo-hopenhayn/',
 'https://economics.ucla.edu/person/andrew-atkeson/',
 'https://economics.ucla.edu/person/david-baqaee/',
 'https://economics.ucla.edu/person/saki-bigio/',
 'https://economics.ucla.edu/person/dora-costa/',
 'https://economics.ucla.edu/person/pablo-fajgelbaum/',
 'https://economics.ucla.edu/person/francois-geerolf/',
 'https://economics.ucla.edu/person/jay-lu/',
 'https://economics.ucla.edu/person/andres-santos/',
 'https://economics.uc

### Q3.B.) Webcrawl the links from A and use RegEx to get all the emails and phone numbers of ladder faculty profiles

In [24]:
# Set Regex
phone_regex = r"\W\d{3}\W \d{3}[-]?\d{4}"
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+[A-Za-z0-9.-]?+\.[A-Z|a-z]{2,}\b'


prof_info = []

for url in prof_urls:
    html = requests.get(url, headers = headers).content
    soup = BeautifulSoup(html,"html.parser")
    
    # Name
    name = soup.find("h1", style=True).text
    
    # Phone
    phone = np.array(re.findall(phone_regex,soup.prettify()))
    if phone is not None:
        phone = phone
    else:
        phone = "NAN"
        
    # Email
    email = soup.find("a", href = re.compile(r'ucla.edu'),title = re.compile(r'ucla.edu'))
    if email is not None:
        email = email.text
    else:
        email = "NAN"
        
    # Append
    prof_info.append({"Name":name,"Phone":phone,"Email":email})
    
prof_df = pd.DataFrame(prof_info)

In [25]:
prof_df

,Name,Phone,Email
0,Adriana Lleras-Muney,[(310) 825-3925],alleras@econ.ucla.edu
1,Oleg Itskhoki,[],itskhoki@econ.ucla.edu
2,Rosa Matzkin,[(310) 825-7371],matzkin@econ.ucla.edu
3,Alexander Bloedel,[],abloedel@econ.ucla.edu
4,Denis Chetverikov,[(310) 825-4521],chetverikov@econ.ucla.edu
5,Lee E. Ohanian,[(310) 825-0979],ohanian@econ.ucla.edu
6,Bernardo S. Silveira,[],silveira@econ.ucla.edu
7,Michela Giorcelli,[(310) 825-1011],mgiorcelli@econ.ucla.edu
8,Simon Board,[(310) 825-5304],sboard@econ.ucla.edu
9,Hugo Hopenhayn,[(310) 206-8896],hopen@econ.ucla.edu


# Q4.) Selenium

### Q4.A.) Pick a website that has useful data to a business or economic question. Put your website you plan to scrape here : https://docs.google.com/spreadsheets/d/1yZTCktXhj7vT40HsITRQI_9FFZ-QIovSv4UZP9w67iI/edit?usp=sharing
### You must have use website that no other group has. First come first serve 

In [27]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import warnings
warnings.filterwarnings("ignore")

In [28]:
URL = "https://www.apartmentfinder.com/"
PATH = "/Applications/Google Chrome.app"

### Q4.B.) Use Selenium to scrape valuable information from your website and store in a dataframe.

In [105]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# set up the webdriver
driver = webdriver.Chrome(PATH)
driver.get(URL)


# find the search bar and enter a search term
search_box = driver.find_element('id','autocompleteInput')
search_box.send_keys("Westwood, Los Angeles, CA")

# find the search button and click it
search_box.send_keys(Keys.RETURN)

# Wait for the page to load and find the list of properties
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.XPATH, '//span[@data-item="name"]')))

# Extract the desired information from the search results
properties = driver.find_elements(By.XPATH, '//span[@data-item="name"]')

# Create empty lists to store the data
names = []
addresses = []
prices = []
beds = []

# Loop through each property and extract the data
for property in properties:
    name = property.find_elements(By.XPATH, '//span[@data-item="name"]')
    for n in name:
        n = n.text
        names.append(n)
        
    address = property.find_elements(By.XPATH,"//address[@class='flex-12 address ellipses']")
    for a in address:
        a = a.text
        addresses.append(a)
        
    price = property.find_elements(By.XPATH,'//span[@class="altRentDisplay layout-hidden-xs"]')
    for p in price:
        p = p.text
        prices.append(p.strip())
        
    bed = property.find_elements(By.XPATH,'//span[@class="unitLabel"]')
    for b in bed:
        b = b.text
        beds.append(b)


names = [n for n in names if n != '']
# Create a Pandas DataFrame with the data
data = {
    'Name':names,
    'Address': addresses,
    'Price': prices,
    'Beds': beds
}

df = pd.DataFrame(data)

# Close the driver
driver.quit()

In [132]:
df = df.drop_duplicates()
print(df)

                                  Name  \
0                     Archer Hollywood   
1   El Centro Apartments and Bungalows   
2                   Eastown Apartments   
3                           The Medici   
4                          The Vermont   
5                   Vision on Wilshire   
6                          The Q Playa   
7                    Hanover Hollywood   
8                                &Pico   
9                         Palazzo West   
10                                 VOX   
11                         Atlas House   
12                            Ferrante   
13                      Kinley West LA   
14      Lido Apartments - 4847 Oakwood   
15                                 ARQ   
16             Residences at West Edge   
17      Private Suite - Co-living Life   
18                              Jasper   
19                     Ardence & Bloom   
20                    Violet on Virgil   
21                               Linea   
22            The Landmark Los Ang

### Q4.C.) Write a short paragraph about the businesses or research that would use the data you scraped. Descrivbe it's value and what it can be used for.